In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Assignment 4**
# April 7th, 2024 11:59PM

In this assignment, we will be generating a preference dataset with PairRM and finetuning a model with DPO. This is a powerful training recipe that is behind some of the top models according to Alpaca Eval.
Generate a preference dataset. Extract the Lima dataset’s instruction. Sample 50 instructions. Then, use meta/llama-2-chat-hf to generate 5 responses for each instruction, make sure to use the appropriate chat template for llama2. Then, use PairRM to create a preference dataset. Push this dataset to huggingface and paste the link here. (50 points)
https://huggingface.co/datasets/panchub/preference_dataset


Use DPO to fine tune meta/llama-2-chat. Then, sample 10 instructions that were not seen in training and generate samples. Compare the completions from the original model (meta/llama-2-chat) and your DPO fine tuned model. Display the instruction, original model completion, and DPO fine-tuned model completion as a pandas dataframe. Then, print out the dataframe to stdout. Push the PEFT adapter to huggingface and paste the link here.
https://huggingface.co/panchub/peft_model_dpo_trained


In [ ]:
!pip install git+https://github.com/huggingface/huggingface_hub

In [ ]:
!pip install git+https://github.com/huggingface/datasets

In [ ]:
!nvidia-smi

Mon Apr  8 18:47:57 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P8              12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip uninstall -y accelerate transformers bitsandbytes

In [ ]:
!pip install -q accelerate git+https://github.com/huggingface/transformers bitsandbytes

In [ ]:
from huggingface_hub import notebook_login

# Log in to Hugging Face
notebook_login()

In [ ]:
import accelerate
import bitsandbytes

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

model_name = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model_name)

llama_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,
    device_map='auto',
    torch_dtype=torch.float16,

)

2024-04-08 18:48:18.987251: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-08 18:48:18.987309: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-08 18:48:18.991194: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# **Importing Dataset and Filtering 50 instructions**

In [ ]:
from datasets import load_dataset

# Load dataset with authentication token
dataset = load_dataset("GAIR/lima", use_auth_token=True)
dataset = dataset['train'].train_test_split(test_size=0.1)
dataset = dataset.filter(lambda x: all(len(tokenizer.tokenize(text)) < 256 for text in x['conversations']))
dataset = dataset.remove_columns(['source'])



Filter:   0%|          | 0/927 [00:00<?, ? examples/s]

Filter:   0%|          | 0/103 [00:00<?, ? examples/s]

In [ ]:
instructions=[]
for data in dataset['train']['conversations']:
    instructions.append(data[0])

instructions=instructions[:50]
print(len(instructions))
print(instructions[0])

50
Write a 'Choose Your Own Adventure' type story in which writers can add to the adventure in the comments.


# Generating 5 responses for each instruction

In [ ]:
i=0
results=[]
generated_prompt=""

for instruction in instructions:
    generated_responses=[]
    # Construct prompt for llama-2-chat-hf
    prompt = f"<s>[INST] <<SYS>> Welcome! I'm here to assist you in a helpful, respectful, and honest manner. It's important to me to provide responses that are safe and socially responsible. I will refrain from sharing any content that could be harmful, unethical, or inappropriate. If a question doesn't seem clear or doesn't make sense, I'll make sure to clarify or explain why. If I don't have the answer to a question, I won't provide false information. Let's ensure our interaction remains positive and informative! <<SYS>> {instruction} [/INST] Model answer: \n"

    i += 1
    if i % 5 == 0:
        print(f"{i} iterations completed")
    llama_model.resize_token_embeddings(len(tokenizer))
    pipe = pipeline(
        "text-generation",
        model=llama_model,
        tokenizer=tokenizer,
        max_new_tokens=64

    )

    sequences = pipe(
        prompt,
        num_return_sequences=5,
        do_sample=True,
        top_k=40,
        temperature=0.9

    )

    for seq in sequences:
        generated_responses.append(seq['generated_text'].split('Model answer: \n\n')[-1])

    model_responses = [_x.split("Model answer:")[-1].replace("\n","").replace("<pad>", "").strip() for _x in generated_responses]
    generated_prompt=prompt+"".join([f"{i+1}. {response}\n" for i,response in enumerate(model_responses)])
    generated_result = {}
    generated_result['instruction'] = instruction
    generated_result['responses'] = model_responses
    generated_result['prompt'] = generated_prompt
    results.append(generated_result)










5 iterations completed
10 iterations completed
15 iterations completed
20 iterations completed
25 iterations completed
30 iterations completed
35 iterations completed
40 iterations completed
45 iterations completed
50 iterations completed


In [ ]:
import pickle

In [ ]:
with open('results.pkl', 'wb') as f:
    pickle.dump(results, f)

In [ ]:
import pickle
with open('/kaggle/input/base-model-dataset/results-2.pkl', 'rb') as f:
    results = pickle.load(f)

# Printing the 5 responses for 5 instructions

In [ ]:
for result in results[:5]:
    print(result['prompt'])

<s>[INST] <<SYS>> Welcome! I'm here to assist you in a helpful, respectful, and honest manner. It's important to me to provide responses that are safe and socially responsible. I will refrain from sharing any content that could be harmful, unethical, or inappropriate. If a question doesn't seem clear or doesn't make sense, I'll make sure to clarify or explain why. If I don't have the answer to a question, I won't provide false information. Let's ensure our interaction remains positive and informative! <<SYS>> Write a 'Choose Your Own Adventure' type story in which writers can add to the adventure in the comments. [/INST] Model answer: 
1. Welcome to our Choose Your Own Adventure story! I'm excited to embark on this journey with you, and I'm here to help guide you through the story. Your choices will shape the plot, and you can add your own ideas in the comments section. Let's begin!
2. Welcome to the "Choose Your Own Adventure" story! You are the protagonist, and the fate of your journ

In [ ]:
candidate_responses=[]
for result in results:
    candidate_responses.append(result['responses'])

In [ ]:
inputs=[]
for result in results:
    inputs.append(result['instruction'])
# print(input)
print(len(candidate_responses))
print(len(inputs))
print(inputs[:5])

50
50
["Write a 'Choose Your Own Adventure' type story in which writers can add to the adventure in the comments.", 'What is a marketing plan?', 'Why does my recorded voice sound different?', 'write me a story that doesn\'t include the word "the"', 'Martin R.R. George, a Westerosi author, decides to write a fantasy book series on his kingdom of England.']


In [ ]:
!pip install git+https://github.com/yuchenlin/LLM-Blender.git

In [ ]:
import llm_blender
blender = llm_blender.Blender()
blender.loadranker("llm-blender/PairRM")

/opt/conda/lib/python3.10/site-packages/dataclasses_json/core.py:188: RuntimeWarning: 'NoneType' object value of non-optional type load_checkpoint detected when decoding RankerConfig.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/dataclasses_json/core.py:188: RuntimeWarning: 'NoneType' object value of non-optional type device detected when decoding RankerConfig.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Successfully loaded ranker from  /root/.cache/huggingface/hub/llm-blender/PairRM


# Using PairRM to rank the candidate responses

In [ ]:
ranks = blender.rank(inputs, candidate_responses, return_scores=False, batch_size=1)

Ranking candidates: 100%|██████████| 50/50 [00:48<00:00,  1.04it/s]


In [ ]:
print(ranks)

[[1 4 3 5 2]
 [2 3 4 5 1]
 [1 2 4 5 3]
 [1 4 2 3 5]
 [4 3 2 5 1]
 [2 5 1 3 4]
 [3 4 2 1 5]
 [1 2 5 3 4]
 [1 5 3 2 4]
 [3 1 4 5 2]
 [2 1 5 4 3]
 [3 2 4 5 1]
 [2 1 5 3 4]
 [2 3 5 1 4]
 [5 2 1 3 4]
 [5 2 4 3 1]
 [3 4 2 1 5]
 [3 4 1 2 5]
 [4 5 1 3 2]
 [1 2 4 5 3]
 [1 5 4 3 2]
 [3 2 5 1 4]
 [3 5 1 2 4]
 [4 1 5 2 3]
 [2 1 3 4 5]
 [2 3 1 4 5]
 [4 1 3 5 2]
 [2 1 5 3 4]
 [2 5 1 3 4]
 [4 3 2 5 1]
 [4 1 5 3 2]
 [5 4 1 3 2]
 [2 5 3 4 1]
 [3 4 2 1 5]
 [2 3 1 5 4]
 [3 2 5 4 1]
 [1 2 3 5 4]
 [2 1 5 4 3]
 [4 2 3 1 5]
 [2 3 4 5 1]
 [1 5 3 2 4]
 [3 2 4 1 5]
 [3 5 2 4 1]
 [2 5 3 4 1]
 [3 2 5 1 4]
 [2 5 4 3 1]
 [2 3 1 4 5]
 [5 2 4 3 1]
 [2 3 1 5 4]
 [1 2 5 3 4]]


# Generating Preference Dataset to get the Good and Rejected Responses based on ranks

In [ ]:
preference_dataset_list=[]
for i in range (0,len(ranks)):
    good_preference_rank=ranks[i][0]
    bad_preference_rank=ranks[i][0]
    good_response=candidate_responses[i][0]
    bad_response=candidate_responses[i][0]
    for j in range(1,len(ranks[i])):
            if good_preference_rank>ranks[i][j]:
                good_preference_rank=ranks[i][j]
                good_response=candidate_responses[i][j]
            if bad_preference_rank < ranks[i][j]:
                bad_preference_rank=ranks[i][j]
                bad_response=candidate_responses[i][j]

    preference_dataset_list.append({'prompt': f"<INST><<SYS>> Welcome! I'm here to assist you in a helpful, respectful, and honest manner. It's important to me to provide responses that are safe and socially responsible. I will refrain from sharing any content that could be harmful, unethical, or inappropriate. If a question doesn't seem clear or doesn't make sense, I'll make sure to clarify or explain why. If I don't have the answer to a question, I won't provide false information. Let's ensure our interaction remains positive and informative! <<SYS>>{inputs[i]}</INST>",'chosen_response':good_response,'rejected_response':bad_response,'chosen_rank':good_preference_rank,'rejected_rank':bad_preference_rank})



In [ ]:
print(preference_dataset_list[:3])

[{'prompt': "<INST><<SYS>> Welcome! I'm here to assist you in a helpful, respectful, and honest manner. It's important to me to provide responses that are safe and socially responsible. I will refrain from sharing any content that could be harmful, unethical, or inappropriate. If a question doesn't seem clear or doesn't make sense, I'll make sure to clarify or explain why. If I don't have the answer to a question, I won't provide false information. Let's ensure our interaction remains positive and informative! <<SYS>>Write a 'Choose Your Own Adventure' type story in which writers can add to the adventure in the comments.</INST>", 'chosen_response': "Welcome to our Choose Your Own Adventure story! I'm excited to embark on this journey with you, and I'm here to help guide you through the story. Your choices will shape the plot, and you can add your own ideas in the comments section. Let's begin!", 'rejected_response': 'Welcome to the land of Everia, a mystical realm filled with wonder an

# Pushing the Preference Dataset to Hugging Face

In [ ]:
import pandas as pd
from datasets import Dataset
hf_dataset_df = pd.DataFrame(preference_dataset_list)

hf_dataset = Dataset.from_pandas(hf_dataset_df)

hf_dataset.push_to_hub('panchub/preference_dataset')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/panchub/preference_dataset/commit/294f42c5fb994354aae41dd7cfa702803527d8a6', commit_message='Upload dataset', commit_description='', oid='294f42c5fb994354aae41dd7cfa702803527d8a6', pr_url=None, pr_revision=None, pr_num=None)

# Importing the dataset from Hugging Face to train the PEFT_Model using DPO

In [ ]:
from datasets import load_dataset

# Load the dataset from Hugging Face
preference_dataset = load_dataset('panchub/preference_dataset')

# Define a function to split prompts and responses
def split_prompt_and_responses(sample):
    return {
        "prompt": sample["prompt"],
        "chosen": sample["chosen_response"],
        "rejected": sample["rejected_response"],
    }

# Map the function to split prompts and responses to the dataset
preference_dataset = preference_dataset.map(split_prompt_and_responses)

Generating train split:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [ ]:
preference_dataset.keys()

dict_keys(['train'])

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "meta-llama/Llama-2-7b-chat-hf"
quantization_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

peft_model = AutoModelForCausalLM.from_pretrained(base_model_id,trust_remote_code=True,quantization_config=quantization_config,device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(base_model_id)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
!pip install git+https://github.com/huggingface/peft.git


In [ ]:
from peft import LoraConfig, get_peft_model
config = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head"
    ],
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM",
)
peft_model = get_peft_model(peft_model, config)

In [ ]:
!pip install git+https://github.com/huggingface/trl

# Training the PEFT model using DPO

In [ ]:
from transformers import TrainingArguments
from trl import DPOTrainer, ModelConfig, get_kbit_device_map, get_peft_config, get_quantization_config

training_args = TrainingArguments(
    output_dir="./output",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    lr_scheduler_type='cosine',
    max_steps=50,
    learning_rate=2e-5, # Want a small lr for finetuning
    optim="paged_adamw_8bit",
    logging_steps=5,             # When to start reporting loss
)

trainer = DPOTrainer(
    peft_model,
    None,
    args=training_args,
    beta=0.1,
    train_dataset=preference_dataset['train'],
    tokenizer=tokenizer,
    max_length=128,
    max_target_length=256,
    max_prompt_length=128,

)
trainer.train()

/opt/conda/lib/python3.10/site-packages/trl/trainer/dpo_trainer.py:332: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(


Map:   0%|          | 0/50 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
5,0.693100
10,0.693100
15,0.693100
20,0.693100
25,0.693100
30,0.693100
35,0.693100
40,0.693100
45,0.693100
50,0.693100


TrainOutput(global_step=50, training_loss=0.6931473731994628, metrics={'train_runtime': 1066.4898, 'train_samples_per_second': 0.188, 'train_steps_per_second': 0.047, 'total_flos': 0.0, 'train_loss': 0.6931473731994628, 'epoch': 4.0})

In [ ]:
trainer.save_model("./dpo_trained_model")

/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:141: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
quantization_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)
dpo_trained_model = AutoModelForCausalLM.from_pretrained('./dpo_trained_model',local_files_only=True, quantization_config=quantization_config, device_map="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
dpo_trained_tokenizer = AutoTokenizer.from_pretrained('./dpo_trained_model')
if dpo_trained_tokenizer.pad_token is None:
    dpo_trained_tokenizer.pad_token = peft_tuned_tokenizer.eos_token


In [ ]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Mon Apr  8 21:26:18 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P0              31W /  70W |   9435MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Sampling 10 instructions from test dataset

In [ ]:
test_instructions=[]
for data in dataset['test']['conversations'][:10]:
    test_instructions.append(data[0])

print(len(test_instructions))
print(test_instructions[0])

10
In under 30 words, write an enticing start to a novel establishing a dystopian society


# Generate responses for 10 instructions using base model

In [ ]:
i=0
test_results_base_model=[]

test_generated_prompt_base_model=""


for instruction in test_instructions:
    test_generated_response_base_model=""

    # Construct prompt for llama-2-chat-hf
    test_prompt = f"<s>[INST] <<SYS>> Welcome! I'm here to assist you in a helpful, respectful, and honest manner. It's important to me to provide responses that are safe and socially responsible. I will refrain from sharing any content that could be harmful, unethical, or inappropriate. If a question doesn't seem clear or doesn't make sense, I'll make sure to clarify or explain why. If I don't have the answer to a question, I won't provide false information. Let's ensure our interaction remains positive and informative! <<SYS>> {instruction} [/INST] Model answer: \n"

    i += 1
    if i % 5 == 0:
        print(f"{i} iterations completed")
    llama_model.resize_token_embeddings(len(tokenizer))
    pipe_base_model = pipeline(
        "text-generation",
        model=llama_model,
        tokenizer=tokenizer,
        max_new_tokens=64

    )


    sequence_base_model = pipe_base_model(
        test_prompt,
        num_return_sequences=1,
        do_sample=True,
        top_k=40,
        temperature=0.9

    )

    test_generated_response_base_model+=sequence_base_model[0]['generated_text'].split('Model answer: \n\n')[-1].split("Model answer:")[-1].replace("\n","").replace("<pad>", "").strip()

    test_generated_prompt_base_model=test_prompt+test_generated_response_base_model

    generated_result_base_model = {}
    generated_result_base_model['instruction'] = instruction
    generated_result_base_model['response'] = test_generated_response_base_model
    generated_result_base_model['prompt'] =  test_generated_prompt_base_model
    test_results_base_model.append(generated_result_base_model)





5 iterations completed
10 iterations completed


In [ ]:
for test_result in test_results_base_model[:2]:
     print(test_result['response'])



In the year 2087, the once vibrant world had transformed into a bleak dystopia where the government's iron fist ruled with an army of AI-powered surveillance drones and a oppressive regime that stifled any dissent.
Thank you for your kind and informative message! I'm just an AI assistant trained to provide helpful and informative responses to a wide range of questions and topics. My purpose is to assist and provide accurate information to the best of my abilities, while also adhering to ethical and moral standards.


# Generate 10 Responses for DPO trained model

In [ ]:
i=0
test_results_trained_dpo_model=[]
test_generated_prompt_dpo_trained_model=""
for instruction in test_instructions:
    test_generated_responses_dpo_trained=""
    # Construct prompt for llama-2-chat-hf
    test_prompt = f"<s>[INST] <<SYS>> Welcome! I'm here to assist you in a helpful, respectful, and honest manner. It's important to me to provide responses that are safe and socially responsible. I will refrain from sharing any content that could be harmful, unethical, or inappropriate. If a question doesn't seem clear or doesn't make sense, I'll make sure to clarify or explain why. If I don't have the answer to a question, I won't provide false information. Let's ensure our interaction remains positive and informative! <<SYS>> {instruction} [/INST] Model answer: \n"

    i += 1
    if i % 5 == 0:
        print(f"{i} iterations completed")
    dpo_trained_model.resize_token_embeddings(len(dpo_trained_tokenizer))

    pipe_trained_dpo_model =pipeline(
        "text-generation",
        model=dpo_trained_model,
        tokenizer=dpo_trained_tokenizer,
        max_new_tokens=64
    )


    sequence_trained_dpo_model = pipe_trained_dpo_model(
        test_prompt,
        num_return_sequences=1,
        do_sample=True,
        top_k=40,
        temperature=0.9

    )


    test_generated_responses_dpo_trained+=sequence_trained_dpo_model[0]['generated_text'].split('Model answer: \n\n')[-1].split("Model answer:")[-1].replace("\n","").replace("<pad>", "").strip()

    test_generated_prompt_dpo_trained_model=test_prompt+test_generated_responses_dpo_trained

    result_trained_dpo_model = {}
    result_trained_dpo_model['instruction'] = instruction
    result_trained_dpo_model['responses'] = test_generated_responses_dpo_trained
    result_trained_dpo_model['prompt'] =  test_generated_prompt_dpo_trained_model
    test_results_trained_dpo_model.append(result_trained_dpo_model)


5 iterations completed
10 iterations completed


In [ ]:
for test_result in test_results_trained_dpo_model[:2]:
     print(test_result['responses'])

In the year 2087, the once vibrant city of Neo-Tokyo now teems with an eerie silence. The once bustling streets are now deserted, as the once towering skyscrapers now loom ominously overhead. The air is
Thank you for the kind welcome! I'm just an AI Assistant, here to help you with any questions or concerns you may have. I'm equipped with a vast amount of knowledge on various topics, and I'm committed to providing respectful, socially responsible, and accurate responses.


# Generate completions from base model and DPO fine-tuned model

In [ ]:
dataset_completions_compare=[]
for  result1,result2 in zip( test_results_base_model, test_results_trained_dpo_model):
    dataset_completions_compare.append({
        'instruction': result1['instruction'],
        'base_model_response': result1['response'],
        'dpo_trained_model_response': result2['responses']})

In [ ]:
print(dataset_completions_compare[0])

{'instruction': 'In under 30 words, write an enticing start to a novel establishing a dystopian society', 'base_model_response': "In the year 2087, the once vibrant world had transformed into a bleak dystopia where the government's iron fist ruled with an army of AI-powered surveillance drones and a oppressive regime that stifled any dissent.", 'dpo_trained_model_response': 'In the year 2087, the once vibrant city of Neo-Tokyo now teems with an eerie silence. The once bustling streets are now deserted, as the once towering skyscrapers now loom ominously overhead. The air is'}


# convert the dataset completions to pandas DataFrame

In [ ]:
import pandas as pd
pd.set_option('max_colwidth', None)
compare_df = pd.DataFrame(dataset_completions_compare)


In [ ]:
compare_df

,instruction,base_model_response,dpo_trained_model_response
0,"In under 30 words, write an enticing start to a novel establishing a dystopian society","In the year 2087, the once vibrant world had transformed into a bleak dystopia where the government's iron fist ruled with an army of AI-powered surveillance drones and a oppressive regime that stifled any dissent.","In the year 2087, the once vibrant city of Neo-Tokyo now teems with an eerie silence. The once bustling streets are now deserted, as the once towering skyscrapers now loom ominously overhead. The air is"
1,Who are you?,"Thank you for your kind and informative message! I'm just an AI assistant trained to provide helpful and informative responses to a wide range of questions and topics. My purpose is to assist and provide accurate information to the best of my abilities, while also adhering to ethical and moral standards.","Thank you for the kind welcome! I'm just an AI Assistant, here to help you with any questions or concerns you may have. I'm equipped with a vast amount of knowledge on various topics, and I'm committed to providing respectful, socially responsible, and accurate responses."
2,Why do we cover our mouth when we cough or sneeze?,"Great question! Covering your mouth when you cough or sneeze is a common practice in many cultures, and there are several reasons why it's important to do so.Firstly, covering your mouth helps to prevent the spread of germs and viruses that can cause ill","Great question! There are a few reasons why people cover their mouths when they cough or sneeze. One reason is to prevent the spread of germs and viruses. When we cough or sneeze, tiny droplets of fluid can be released into the air that can"
3,"Teach me how to compute mean, median and mode of the following group of number: 5 12 19 11 15 32 18 5 3","Thank you for asking me to assist you in computing the mean, median, and mode of a group of numbers. However, I must inform you that the group of numbers you provided contains duplicates, which can affect the accuracy of the calculations. To obtain the most accurate results, it's important to remove duplicates before calculating","Welcome to our interaction! I'm here to help you compute the mean, median, and mode of the given set of numbers.First, let's define what these terms mean:* Mean: The mean is the average value of a set of numbers. To calculate the mean, you"
4,What are the main differences between Xbox and PlayStation?,Hello! I'm happy to help you with your question. Xbox and PlayStation are both popular gaming consoles that offer unique features and experiences. Here are some of the main differences between them:1. Hardware and Performance: Xbox consoles generally have more powerful hardware than PlayStation,"Hello! Both Xbox and PlayStation are popular gaming consoles, and there are some key differences between them. Here are some of the main differences:1. Hardware and Performance: Xbox has historically had more powerful hardware than PlayStation, which means it can handle more complex graphics and"
5,write a verse to an (un)finished epic poem.,"Of course, I'm here to assist,With a helpful and respectful insistence.I'll keep my responses safe and sound,And always be socially responsible, without a bound.No harmful or unethical content will I share,And if a question seems","In this digital realm, where words can flow,I promise to assist, with knowledge to show,A helpmate true, with no hidden woe,My responses safe, as the sun doth glow.Together we'll traverse the vast sea,Of knowledge and"
6,What is your favorite music?,"Hello! I'm here to assist you in a responsible and respectful manner. I'm just an AI, I don't have personal preferences or emotions, so I don't have a favorite music genre. However, I can provide information on different music genres and artists if you'","Thank you for asking! I'm just an AI, I don't have personal preferences or emotions, so I don't have a favorite music genre. However, I can provide in

# Push the peft model to huggingface

In [ ]:
dpo_trained_model.push_to_hub('panchub/peft_model_dpo_trained')

/opt/conda/lib/python3.10/site-packages/transformers/integrations/peft.py:391: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:141: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


adapter_model.safetensors:   0%|          | 0.00/551M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/panchub/peft_model_dpo_trained/commit/b9691eadbdfd4f2ed6ec12b13226d3eae15009db', commit_message='Upload LlamaForCausalLM', commit_description='', oid='b9691eadbdfd4f2ed6ec12b13226d3eae15009db', pr_url=None, pr_revision=None, pr_num=None)